In [25]:
struct MyRange{T<:Number} 
    start:: T
    step:: T
    stop:: T
end

In [2]:

a = MyRange(1,2,20)
a

MyRange{Int64}(1, 2, 20)

In [3]:
function _MyRange(a, i)
    (1<= i <= 1+(a.stop-a.start)//a.step) && (res = a.start + (i-1)*a.step)
    res
end

_MyRange (generic function with 1 method)

In [9]:
_MyRange(a,5)

9

In [13]:
Base.getindex(a::MyRange, i::Int) = _MyRange(a,i)
a[5]

9

In [ ]:
struct LazyLinSpace
    start:: Float64
    stop:: Float64
    n:: Int64
end

function Base.getindex(a::MyLinSpace,i::Int)
    dx = (a.stop-a.start)/a.n
    a.start + dx*(i-1)
end



In [43]:
function interpolate(a, b, x)
    if x <0 && x>1
        println("Out of interpolation range")
    end
    (1-x)*a + x*b
end

mutable struct UnitStepRange{T<:Number} 
    start:: T
    step:: T
    stop:: T
end

Base.getindex(r::UnitStepRange, i::Int64) = _MyRange(r,i)

# (r::StepRange)(i::Float64) = interpolate(r[floor(i)], r[floor(i)+1], i-floor(i))
(r::UnitStepRange)(i::Float64) = interpolate(r[floor(Int, i)], r[floor(Int, i)+1], i-floor(Int, i))

r = UnitStepRange(1,2,20)
println(typeof(r))
r(1.5)

UnitStepRange{Int64}


2.0

In [35]:
mutable struct CallTest
    a::Float64 
end

(c::CallTest)(b::Float64) = c.a*b

c = CallTest(2.0)
c(3.0)

6.0

In [49]:
struct StrangMatrix 
    n
end

function myfunc(A::StrangMatrix, x::Vector)
    y = similar(x)
    n = length(x)
    y[1] = -2x[1] + x[2]
    y[n] = -2x[n] + x[n-1]
    for i in 2:n-1
        y[i] = x[i-1] - 2x[i] + x[i+1]
    end
    y
end

# (A::StrangMatrix)(x::Vector) = myfunc(A, x)
Base.:*(A::StrangMatrix,B::Vector) = myfunc(A, x)

A = StrangMatrix(4)
x = ones(10)

println(x)
# myfunc(A, x)
A*x

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


10-element Vector{Float64}:
 -1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -1.0